In [ ]:
# 一、导入必要的库
# requests：用于发送 HTTP 请求，以便从网页获取数据。
import requests
# BeautifulSoup：是一个用于解析 HTML 和 XML 文档的库，能够方便地从网页内容中提取所需信息。
from bs4 import BeautifulSoup
import time
import pandas as pd

# 二、设置请求头
# 设置了一个请求头（headers），其中包含了User-Agent信息。User-Agent是浏览器在向服务器发送请求时表明自身身份的标识。
# 通过设置合适的User-Agent，可以让服务器认为请求是来自正常的浏览器访问，而不是脚本程序，从而提高请求的成功率，避免一些网站对非浏览器请求的限制。
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
}

# 三、定义获取歌曲信息的函数get_info
# 这个函数接受一个网页链接（url）作为参数，用于从指定的酷狗音乐排行榜页面获取歌曲的相关信息，具体步骤如下：
# 首先使用requests.get()方法发送一个 HTTP GET 请求到指定的url，并带上之前设置好的请求头（headers），获取网页的原始内容，并将其赋值给web_data。
# 然后使用BeautifulSoup库对获取到的网页内容（web_data.text）进行解析，指定解析器为lxml，得到一个可以方便进行数据提取的对象soup。
# 接着通过BeautifulSoup的select方法，根据特定的 CSS 选择器从解析后的网页内容中分别提取出歌曲的排名信息（ranks）、歌曲名称和歌手信息（titles）以及歌曲时长信息（times）。
# 之后通过循环遍历同时从ranks、titles和times中取出对应的数据，并进行一些数据清洗和整理操作，将歌曲的排名、歌手、歌曲名称和时长信息组合成一个字典（data），并添加到data_list列表中。
# 最后返回包含所有歌曲信息字典的列表data_list。
def get_info(url):
    web_data = requests.get(url, headers=headers)
    soup = BeautifulSoup(web_data.text, 'lxml')

    ranks = soup.select('span.pc_temp_num')
    titles = soup.select('div.pc_temp_songlist > ul > li > a')
    times = soup.select('span.pc_temp_tips_r > span')
    
    data_list = []
    for rank, title, time in zip(ranks, titles, times):
        data = {
            "排名": rank.get_text().strip(),
            "歌手": title.get_text().replace("\n", "").replace("\t", "").split('-')[1],
            "歌曲": title.get_text().replace("\n", "").replace("\t", "").split('-')[0],
            "时长": time.get_text().strip()
        }
        data_list.append(data)
    
    return data_list


# 四、主程序部分
# 首先，通过列表推导式生成了一个包含 23 个酷狗音乐排行榜页面链接的列表（urls）。每个链接对应酷狗音乐排行榜的不同页面，通过格式化字符串的方式，将页码i（从 1 到 23）插入到链接中，形成完整的排行榜页面链接。
# 接着初始化一个空列表all_data，用于存储从各个排行榜页面获取到的所有歌曲信息。
# 然后通过循环遍历urls列表中的每个链接，对于每个链接：
# 调用get_info函数获取该页面上的歌曲信息，并将返回的歌曲信息列表赋值给data。
# 将data中的歌曲信息字典添加到all_data列表中，实现了将所有页面的歌曲信息汇总到一起。
# 在每次请求完一个页面后，调用time.sleep(1)让程序暂停 1 秒钟，这样做是为了避免过于频繁地请求酷狗音乐的服务器，防止被服务器识别为恶意爬虫而限制访问。
# 最后，使用pandas库的DataFrame类将汇总后的所有歌曲信息（all_data）转换为一个数据框（df），并通过df.to_excel()方法将数据框中的数据导出为一个名为酷狗音乐Top500排行榜.xlsx的 Excel 文件，
# 其中设置index=False表示不将数据框的行索引导出到 Excel 文件中。
if __name__ == '__main__':
    urls = ["https://www.kugou.com/yy/rank/home/{}-8888.html".format(str(i)) for i in range(1, 24)]
    all_data = []
    for url in urls:
        data = get_info(url)
        all_data.extend(data)
        time.sleep(1)
    df = pd.DataFrame(all_data)
    df.to_excel('酷狗音乐Top500排行榜.xlsx', index=False)
